Learning [here][1]
and [here][2]


  [1]: https://stackoverflow.com/questions/36065646/xgboost-attributeerror-dmatrix-object-has-no-attribute-handle
  [2]: https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-sberbank

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

%matplotlib inline


pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)


# Any results you write to the current directory are saved as output.

In [ ]:
tn = pd.read_csv("../input/train.csv",  parse_dates=['timestamp'])
tt = pd.read_csv("../input/test.csv",  parse_dates=['timestamp'])
macro_df = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

tn = pd.merge(tn, macro_df, how ='left', on='timestamp')
tt = pd.merge(tt, macro_df, how ='left', on='timestamp')

In [ ]:
tn.head()

Lets's get the estimate of shape, dtypes

In [ ]:
print("Shape")
print(tn.shape, tt.shape)
print("dtypes")
print(tn.dtypes)

Looking at the data set, we see lot's of na's. Let's get a count of them

In [ ]:
tn.isnull().sum()

Let's make a mental note to make sure we impute values. For now lets continue.

Let's find correlations in property variables

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ax =tn.price_doc.hist(bins=50)

In [ ]:
ax = np.log(tn['price_doc']).hist(bins=50)

In [ ]:
dtype_df = tn.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

In [ ]:
null_df = tn.isnull().sum().reset_index()
null_df.columns = ["Column", "Null_no"]
null_df=null_df.ix[null_df['Null_no']>0]
ind = np.arange(null_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, null_df.Null_no.values, color='y')
ax.set_yticks(ind)
ax.set_yticklabels(null_df.Column.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()


For Feature Importance, lets use XGB

In [ ]:

from sklearn import model_selection, preprocessing

for o in tn.columns:
    if tn[o].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(tn[o].values)) 
        tn[o] = lbl.transform(list(tn[o].values))
        
tn_Y = tn.price_doc.values
tn_X = tn.drop(["id", "timestamp", "price_doc"], axis = 1)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
dtrain = xgb.DMatrix( tn_X, tn_Y,feature_names=list(tn_X.columns.values))
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=100)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()